This program prepares the data collected for the Healthy Brains project for upload to the NIMH Data Archive (NDA). 

In [1]:
import pandas as pd
import datetime as dt
import numpy as np

# load measures from project data dictionary
measures = ['TEPS', 'MAP-SR', 'CESD', 'COPE', 'CAPE']
dataDict = {m:pd.read_excel('dataPrep/HBP_NDA_DataDict.xlsx', sheet_name=m) for m in measures}

# load pseudo-GUIDs
pGUIDs = pd.read_excel('dataPrep/HBP_NDA_DataDict.xlsx', sheet_name='pseudo-GUIDs') 

# load collected data & sort by 'subnum'
qltrcs_data = pd.read_csv('dataPrep/Healthy+Brains+Project+-+Qualtrics+Survey_April+23,+2020_09.22.csv', skiprows=[1, 2]).sort_values(by='subnum')
intvw_data = None # update when we get the database, remember to sort by subnum
rawData = pd.concat([qltrcs_data, intvw_data], axis=1, sort=False)

[description of Measure class and capabilities]

In [29]:
# creates an object for each measure with the following capabilities
class Measure():
    
    name = None
    NDA_vars = None
    HBP_vars = None
    varmatches = dict([]) 
    Data = None
    
    # matches the HBP and NDA vars in a dictionary
    def matchVars(self):
        HBP = list(self.HBP_vars)
        matches = {v:self.NDA_vars.iloc[HBP.index(v)] for v in HBP}
        return matches
    
    # initializes object instance & sets object variables
    def __init__(self, name):
        global dataDict
        self.name = name
        self.NDA_vars = dataDict[name]['NDA varname']
        self.HBP_vars = dataDict[name]['HBP varname']
        self.varmatches = self.matchVars()
        self.Data = pd.DataFrame(columns=self.NDA_vars)
        return None
    
    # retrieves and returns raw data for a given HBP variable
    def getVarData(self, hbpv):
        global rawData
        d = rawData[hbpv]
        return d
    
    # formats interview_date as closer to upload-ready format, calculates age in months & populates interview_age
    def formatDateObjs(self, df):
        # convert timestamps to datetime objects
        df['interview_date'] = df['interview_date'].apply(lambda x: dt.datetime.strptime(str(x), "%Y-%m-%d %H:%M:%S"))
        df['interview_age'] = df['interview_age'].apply(lambda x: dt.datetime.strptime(str(x), "%m/%d/%Y"))
        # calculate age in months & insert in table
        diffs = df['interview_date'].sub(df['interview_age'])
        df['interview_age'] = diffs.apply(lambda d: round(d.days/30))
        # reformat interview_date as MM/DD/YYYY -- this section seems inefficient, revisit later
        df['interview_date'] = df['interview_date'].apply(lambda x: dt.datetime.strftime(x, "%m/%d/%Y"))
        df['interview_date'] = df['interview_date'].apply(lambda x: dt.datetime.strptime(x, "%m/%d/%Y"))
    
    # adds pseudo-GUIDs
    def addpGUIDs(self, frupld):
        global pGUIDs
        for i, r in frupld.iterrows():
            sid = r['src_subject_id']
            pguid = pGUIDs.query("study_ids == @sid")['pGUIDs'].iloc[0]
            frupld.at[i, 'subjectkey'] = pguid
    
    # adds missing values where appropriate
    def addMissing(self):  
        global dataDict
        placeholders = dataDict[self.name]['missing'].notnull()
        for i, v in placeholders.iteritems():
            if v == False:
                continue
            else:
                ndav = dataDict[self.name].at[i, 'NDA varname']
                mval = dataDict[self.name].at[i, 'missing']
                self.Data[ndav].fillna(mval, inplace=True)
    
    # fills columnns of self.Data with raw data 
    def fillColumns(self):
        for v in self.HBP_vars:
            if isinstance(v, type(float("Nan"))):
                continue
            else:
                data = self.getVarData(v)
                ndav = self.varmatches[v]
                self.Data[ndav] = data
        return None
    
    # recodes sex data as String data instead of Integers
    def fixSexData(self):
        self.Data['sex'].replace({1:'M', 2:'F'}, inplace=True)
        
    def prepDtypeDict(self):
        global dataDict
        types = {'GUID':str, 'String':str, 'Integer':int, 'Float':float}
        dtypes = pd.Series(data=dataDict[self.name]['NDA data type'].to_numpy(), index=dataDict[self.name]['NDA varname'])
        for i, v in dtypes.iteritems():
            if v not in types.keys():
                dtypes.loc[i] = None
            else:    
                dtypes.loc[i] = types[v]
        return dtypes.to_dict()
    
    def setDataTypes(self):
        d = self.prepDtypeDict()
        for v in d.keys(): # "for each NDA var"
            if d[v] == None:
                continue
            elif v == 'visit':
                self.Data[v].replace(np.nan, " ", inplace=True, regex=True)
            else:
                self.Data[v].astype(d[v])

    # applies the methods defined above to produce df of almost-upload-ready data for measure
    def prepData(self):
        self.fillColumns()
        self.formatDateObjs(self.Data) 
        self.addpGUIDs(self.Data)
        self.addMissing()
        self.fixSexData()
        self.setDataTypes()
    
    

[Testing my code with the TEPS]

In [30]:
TEPS = Measure('TEPS')
TEPS.prepData()
TEPS.Data


NDA varname        subjectkey  src_subject_id interview_date  interview_age  \
2            NDAR_INVG0H96HN3           10013     2019-11-19            698   
0            NDAR_INVH5B3AF0L           10014     2019-10-14            698   
13           NDAR_INVW6BGU18M           10017     2020-02-22            704   
1            NDAR_INV17R0Z5L7           10025     2019-10-30            702   
7            NDAR_INV6A044NGU           10029     2020-01-25            700   
3            NDAR_INV3C7NW8HX           10033     2019-12-07            700   
6            NDAR_INVTXVAUVWW           10038     2020-01-22            700   
4            NDAR_INVE96TPXJP           10039     2019-12-14            697   
5            NDAR_INVD3PTWJKW           10040     2019-12-20            723   
11           NDAR_INVFDWK6A6E           10041     2020-02-15            698   
12           NDAR_INVNH8KWFLL           10052     2020-02-19            700   
8            NDAR_INVEXD68KRM           10055     2020-01-27            702   
16           NDAR_INV5X2GD35A           10056     2020-03-14            709   
17           NDAR_INVB56F5JTV           10058     2020-03-16            708   
9            NDAR_INVR0E1GHRN           10060     2020-02-08            706   
10           NDAR_INVYM273ENY           10065     2020-02-12            703   
15           NDAR_INVKDZL1PGT           10075     2020-03-13            704   
14           NDAR_INVK801BF0Z           10082     2020-02-25            708   

NDA varname sex  teps1  teps2  teps3  teps4  teps5  ...  teps18  teps_af_ic  \
2             M      2      4      5      5      6  ...       6         NaN   
0             M      1      6      6      6      6  ...       6         NaN   
13            F      5      6      6      4      4  ...       6         NaN   
1             F      6      6      2      6      4  ...       6         NaN   
7             M      4      6      6      6      2  ...       5         NaN   
3             F      4      6      4      6      2  ...       6         NaN   
6             F      5      6      6      6      4  ...       6         NaN   
4             M      3      6      5      5      2  ...       5         NaN   
5             M      4      5      5      5      4  ...       4         NaN   
11            F      4      4      6      5      1  ...       5         NaN   
12            F      4      6      6      6      3  ...       6         NaN   
8             F      4      6      3      5      1  ...       5         NaN   
16            M      2      4      4      4      2  ...       5         NaN   
17            F      4      6      6      4      3  ...       6         NaN   
9             F      3      6      6      3      6  ...       4         NaN   
10            F      4      6      6      6      4  ...       6         NaN   
15            F      4      6      6      6      2  ...       6         NaN   
14            F      2      5      5      6      5  ...       5         NaN   

NDA varname  teps_cf_ic  teps_total_ic  tepc_acsi  teps_ap  teps_cp  visnum  \
2                   NaN            NaN        NaN      NaN      NaN     NaN   
0                   NaN            NaN        NaN      NaN      NaN     NaN   
13                  NaN            NaN        NaN      NaN      NaN     NaN   
1                   NaN            NaN        NaN      NaN      NaN     NaN   
7                   NaN            NaN        NaN      NaN      NaN     NaN   
3                   NaN            NaN        NaN      NaN      NaN     NaN   
6                   NaN            NaN        NaN      NaN      NaN     NaN   
4                   NaN            NaN        NaN      NaN      NaN     NaN   
5                   NaN            NaN        NaN      NaN      NaN     NaN   
11                  NaN            NaN        NaN      NaN      NaN     NaN   
12                  NaN            NaN        NaN      NaN      NaN     NaN   
8                   NaN            NaN        Na

In [31]:
for v in TEPS.Data.columns:
    print(type(v))
    print(v, ": ", type(TEPS.Data[v][0]))

<class 'str'>
subjectkey :  <class 'str'>
<class 'str'>
src_subject_id :  <class 'numpy.int64'>
<class 'str'>
interview_date :  <class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'str'>
interview_age :  <class 'numpy.int64'>
<class 'str'>
sex :  <class 'str'>
<class 'str'>
teps1 :  <class 'numpy.int64'>
<class 'str'>
teps2 :  <class 'numpy.int64'>
<class 'str'>
teps3 :  <class 'numpy.int64'>
<class 'str'>
teps4 :  <class 'numpy.int64'>
<class 'str'>
teps5 :  <class 'numpy.int64'>
<class 'str'>
teps6 :  <class 'numpy.int64'>
<class 'str'>
teps7 :  <class 'numpy.int64'>
<class 'str'>
teps8 :  <class 'numpy.int64'>
<class 'str'>
teps9 :  <class 'numpy.int64'>
<class 'str'>
teps10 :  <class 'numpy.int64'>
<class 'str'>
teps11 :  <class 'numpy.int64'>
<class 'str'>
teps12 :  <class 'numpy.int64'>
<class 'str'>
teps13 :  <class 'numpy.int64'>
<class 'str'>
teps14 :  <class 'numpy.int64'>
<class 'str'>
teps15 :  <class 'numpy.int64'>
<class 'str'>
teps16 :  <class 'numpy.int64'>
<class

THINGS LEFT TO DO
1. [DONE] convert interview_age to age in months
2. [DONE] add missing values where necessary
3. [DONE] add pseudo-GUIDs
4. [DONE] change sex data from 1s & 2s to Ms & Fs
5. set data types for each variable according to codebook
6. remove vars for which we don't collect data & aren't required (place this before populating sheet if possible)

In writing to CSV remember to set index=False so that it doesn't write the index column.


AFTER WRITING TO CSV:
1. reformat interview_date to MM/DD/YYYY in MS Excel -- first try upload with dates saved as string data and see what happens
2. add first row with label to spreadsheet
3. make sure everything looks good

NOTES::

I still think I can reformat the code to include a dataCleaner class. It could take in the measure.forUpload df, make all the changes, and then return it. Meaning that we will need to set measure.forUpload = dataCleaner.mainFunc(measure)

since there are a handful of data points that copy across measures for each subject, maybe we consider defining a subject class as well, so that we only have to calculate and populate that data once?   

https://nda.nih.gov/vt/

[applying to measures in bulk]

In [ ]:
# eventually, we want this cell, but not right now
Measures = {m:Measure(m) for m in measures}

for M in Measures.values():
    M.fillColumns()

In [ ]:
class DataCleaner():
    
    self.measure = None
    self.df = None
    
    def __init__(self, m):
        self.measure = m
        self.df = m.Data
        return None
    
    def addMissing(self): # KEEP IN MEASURE CLASS
        
    def calcAge(self): 
        
    def formatIntvwDate(self):
        
    def sexData2str(self):
        
    def cleanData(self):

In [ ]:
df = pd.DataFrame(data=[['A', np.dtype(float)], ['B', np.dtype(int)], ['C', np.dtype(str)]], columns=['letters', 'dtypes'])
df2 = pd.DataFrame(data=[['A', float], ['B', int], ['C', str]], columns=['letters', 'dtypes'])
df2

In [ ]:
# some functions to set data types that I haven't completely worked out yet

def prepDtypeDict(self):
    global dataDict
    types = {'GUID':str, 'String':str, 'Integer':int, 'Float':float}
    dtypes = pd.Series(data=dataDict[self.name]['NDA data type'].to_numpy(), index=dataDict[self.name]['NDA varname'])
    for i, r in dtypes.iteritems():
        if r[0] not in types.keys():
            continue
        else:    
            dtypes.loc[i][0] = types[dataDict[i][0]]
    return dtypes.to_dict()
    
def setDataTypes(self):
    d = self.prepDtypeDict()
    for v in self.forUpload.columns: # for each NDA var
        if v not in d.keys():
            continue
        else:
            self.forUpload[v].astype(d[v])

In [ ]:
df2.at[2, 'dtypes']